## P2P loan Risk Analyis and Borrower Risk Profile Presiction- A use case from the LendingClub Loans

Mavis Wong, Yasmin Hassan and Abeba N. Turi

In [14]:
# Import 
from hashlib import sha1
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    cross_val_score,
    train_test_split,
)
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
import altair as alt
import pandas as pd
import numpy as np
# Enable the VegaFusion data transformer
alt.data_transformers.enable("vegafusion")


DataTransformerRegistry.enable('vegafusion')

### Project Overview <br>
_project summary_

### Introduction <br>
_background intro_

### Methods <br>

#### Data <br>

This analysis is based on the historic loan data from Lending club. The key features in this dataset are described as follows:
- purpose: the purpose of the loan
- int.rate: Interest rate of the loan the applicant received
- installment: Monthly payment for the loan the applicant received.
- log.annual.inc: annual_income (growth rate)
- dti: Debt-to-income ratio
- revol.bal': Total credit revolving balance
- revol.util: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
- inq.last.6mths ("The number of inquiries in past 6 months (excluding auto and mortgage inquiries))
- delinq.2yrs: Delinquencies on lines of credit in the last 2 years.
- pub.rec: The number of derogatory public records, which roughly means the number of times the applicant failed to pay(reputation)
- not.fully.paid: binary response on weather the loan is paid or not ("loan_status.....Status of the loan.")
- int.rate: Interest Rate
- installment: Installment amount 
- log.annual.inc- Log of Annual Income
- fico: FICO credit score 
- days.with.cr.line: Days with Credit Line

#### Analysis <br>

_info about model used_

# EDA

In order for us to draw context about the data, let us have a look at the first few rows of the data; check the info about all columns, data types, and number of NaN values; and summary statistis.

In [15]:
# Define the path to the data directory relative to the current notebook
DATA_DIR = '../data'
p2ploan_df = pd.read_csv(f"{DATA_DIR}/loan_data.csv")

p2ploan_df['annual.inc'] = np.exp(p2ploan_df['log.annual.inc']) 
p2ploan_df.head()

,credit.policy,purpose,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid,annual.inc
0,1,debt_consolidation,0.1189,829.10,11.350407,19.48,737,5639.958333,28854,52.1,0,0,0,0,85000.000385
1,1,credit_card,0.1071,228.22,11.082143,14.29,707,2760.000000,33623,76.7,0,0,0,0,65000.000073
2,1,debt_consolidation,0.1357,366.86,10.373491,11.63,682,4710.000000,3511,25.6,1,0,0,0,31999.999943
3,1,debt_consolidation,0.1008,162.34,11.350407,8.10,712,2699.958333,33667,73.2,1,0,0,0,85000.000385
4,1,credit_card,0.1426,102.92,11.299732,14.97,667,4066.000000,4740,39.5,0,1,0,0,80799.999636


In [16]:
p2ploan_df.info
p2ploan_df.isna().any().sum()
p2ploan_df.dtypes

credit.policy          int64
purpose               object
int.rate             float64
installment          float64
log.annual.inc       float64
dti                  float64
fico                   int64
days.with.cr.line    float64
revol.bal              int64
revol.util           float64
inq.last.6mths         int64
delinq.2yrs            int64
pub.rec                int64
not.fully.paid         int64
annual.inc           float64
dtype: object

In [17]:
##Summary Statistics
p2ploan_df.describe()

,credit.policy,int.rate,installment,log.annual.inc,dti,fico,days.with.cr.line,revol.bal,revol.util,inq.last.6mths,delinq.2yrs,pub.rec,not.fully.paid,annual.inc
count,9578.000000,9578.000000,9578.000000,9578.000000,9578.000000,9578.000000,9578.000000,9.578000e+03,9578.000000,9578.000000,9578.000000,9578.000000,9578.000000,9.578000e+03
mean,0.804970,0.122640,319.089413,10.932117,12.606679,710.846314,4560.767197,1.691396e+04,46.799236,1.577469,0.163708,0.062122,0.160054,6.840203e+04
std,0.396245,0.026847,207.071301,0.614813,6.883970,37.970537,2496.930377,3.375619e+04,29.014417,2.200245,0.546215,0.262126,0.366676,6.122753e+04
min,0.000000,0.060000,15.670000,7.547502,0.000000,612.000000,178.958333,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,1.896000e+03
25%,1.000000,0.103900,163.770000,10.558414,7.212500,682.000000,2820.000000,3.187000e+03,22.600000,0.000000,0.000000,0.000000,0.000000,3.850000e+04
50%,1.000000,0.122100,268.950000,10.928884,12.665000,707.000000,4139.958333,8.596000e+03,46.300000,1.000000,0.000000,0.000000,0.000000,5.576400e+04
75%,1.000000,0.140700,432.762500,11.291293,17.950000,737.000000,5730.000000,1.824950e+04,70.900000,2.000000,0.000000,0.000000,0.000000,8.012100e+04
max,1.000000,0.216400,940.140000,14.528354,29.960000,827.000000,17639.958330,1.207359e+06,119.000000,33.000000,13.000000,5.000000,1.000000,2.039784e+06


In [18]:
#Data distribution of selected loan features
numeric_cols_hists = alt.Chart(p2ploan_df).mark_bar().encode(
    alt.X(alt.repeat(), type='quantitative', bin=alt.Bin(maxbins=20)),  
    y='count()'
).properties(
    width=250,
    height=175
).repeat(
    ['installment', 'dti'],  
    columns=3
)

numeric_cols_hists

alt.RepeatChart(...)

In [19]:
# Select only numeric columns
numeric_cols = p2ploan_df.select_dtypes(include='number')

# Calculate the correlation matrix
correlation_matrix = numeric_cols.corr().reset_index().melt('index')
correlation_matrix.columns = ['Variable 1', 'Variable 2', 'Correlation']

# Create a heatmap using Altair
correlation_chart = alt.Chart(correlation_matrix).mark_rect().encode(
    x=alt.X('Variable 1:N', title=''),
    y=alt.Y('Variable 2:N', title=''),
    color=alt.Color('Correlation:Q', scale=alt.Scale(scheme='viridis')),
    tooltip=['Variable 1', 'Variable 2', 'Correlation']
).properties(
    width=400,
    height=400,
    title="Correlation Heatmap"
)

correlation_chart

# The EDA for most of the numerical columns produce no strong general trends. We see a higher correlation level between fico revo.util, fico and interest rate, interest rate and anual income.



alt.Chart(...)

# Research Problem:
Here, from a business perspective, we need to identify high-risk and low-risk loan profiles to improve lending profitability while minimizing default rates. 
This includes assessing borrower behavior, income stability, and loan utilization to create a predictive model for effective risk categorization (see  Coşer et al., 2019 and Khandani et al.,2010).

Key metrix and considerations:
- Debt-to-Income Ratio 
- Credit Utilization Ratio( revol.util)_ how much of their revolving credit borrowers are using relative to their limit with higher values indicating possible financial strain.
- Loan Duration vs. Risk: If longer-term loans are associated with higher default rate (days.with.cr.line).

### Loan categories
Below, to help us create the loan categories, we are using the FICO risk profile categories  [see the Borrower risk profiles categories from here](https://www.consumerfinance.gov/data-research/consumer-credit-trends/student-loans/borrower-risk-profiles/#:~:text=We%20focus%20on%20five%20credit%20score%20levels%3A&text=Subprime%20(credit%20scores%20of%20580,scores%20of%20720%20or%20above)
- Deep subprime (credit scores below 580)
- Subprime (credit scores of 580-619)
- Near-prime (credit scores of 620-659)
- Prime (credit scores of 660-719)
- Super-prime (credit scores of 720 or above)


In [20]:
# Feature Engineering
p2ploan_df['loan_income_ratio'] = (p2ploan_df['installment'] * 12) / p2ploan_df['annual.inc']

# Creating loan Categories
conditions = [
    (p2ploan_df['fico'] >= 720),
    (p2ploan_df['fico'] < 719) & (p2ploan_df['fico'] >= 660),
    (p2ploan_df['fico'] < 659) & (p2ploan_df['fico'] >= 620),
    (p2ploan_df['fico'] < 619) & (p2ploan_df['fico'] >= 580),
    (p2ploan_df['fico'] < 580)
]
loan_categories = ['Super-prime', 'Prime', 'Near-prime', 'Subprime', 'Deep subprime']
p2ploan_df['loan_categories'] = np.select(conditions, loan_categories, default='Unknown')


In [21]:
# Default Rate by Loan Purpose:

# Explode 'purpose' column for analysis
loan_purpose_data = p2ploan_df.explode('purpose')

# Loan Purpose vs Loan Category
purpose_risk_chart = alt.Chart(loan_purpose_data).mark_circle().encode(
    x=alt.X('purpose:N', title='Loan Purpose', sort='-color'),
    y=alt.Y('loan_categories:N', title='loan_categories', sort='color'),
    color=alt.Color('count()', scale=alt.Scale(scheme='viridis'), title='Loan Count'),
    size=alt.Size('count()', title='Loan Count'),
    tooltip=['purpose', 'loan_categories', 'count()']
).properties(
    width=600,
    height=400,
    title='Loan Purpose vs Loan Category'
)

purpose_risk_chart



alt.Chart(...)

### Risk categories
Let us explore the data further with specific borrower risk profile categories 
Based on the above 5 loan categories, we framed three main risk categories as high, medium and low risk profile with: 
fico score of at least 720 (Low Risk), 'fico' score between 650 and 720 ('Medium Risk') and 'fico' score of 650 as 'High Risk'. 

In [22]:
# Creating Risk Categories
conditions = [
    (p2ploan_df['fico'] >= 720),
    (p2ploan_df['fico'] < 720) & (p2ploan_df['fico'] >= 650),
    (p2ploan_df['fico'] < 650)
]
categories = ['Low Risk', 'Medium Risk', 'High Risk']
p2ploan_df['risk_category'] = np.select(conditions, categories, default='Unknown')

categories_hist = alt.Chart(p2ploan_df).mark_bar().encode(
    x=alt.X('risk_category:N', title='Risk Categories'),  
    y=alt.Y('count()', title='Count') \
).properties(
    height=300,
    width=400,
    title="Distribution of Risk Categories"
)

categories_hist
#Notice we have a high concentration of loans in the medium risk category and significant number low risk borrowers as compared to the high risk borrowers.


alt.Chart(...)

In [23]:
#fico by loan purpose
purpose_fico_boxplot = alt.Chart(loan_purpose_data).mark_boxplot().encode(
    x=alt.X('purpose:N', title='Loan Purpose', sort='-y'),  # Categorical on x-axis
    y=alt.Y('fico:Q', title='FICO Score', scale=alt.Scale(domain=[500, 850])),  
    color=alt.Color('purpose:N', legend=None),  
    tooltip=['purpose', 'fico']
).properties(
    width=600,
    height=400,
    title='Boxplot of FICO Scores by Loan Purpose'
)


#Debt to income ratio by risk level
risk_dti_boxplot = alt.Chart(p2ploan_df).mark_boxplot().encode(
    x=alt.X('risk_category:N', title='Risk Category', sort='-y'),  
    y=alt.Y('dti:Q', title='DTI (Debt-to-Income)', scale=alt.Scale(domain=[0, 50])),  
    color=alt.Color('risk_category:N', legend=None),  
    tooltip=['risk_category', 'dti']
).properties(
    width=600,
    height=400,
    title='Boxplot of DTI by Risk Category'
)


purpose_fico_boxplot | risk_dti_boxplot



alt.HConcatChart(...)

From the boxplot above, we see that the low risk borrowers have lower average debt-to-income-ratio as compared to the borrowers with medium and high risk profile, based on their fico score. Note also the outliers in FICO scores for the loan purpose of debt consolidation type.


####  Data Transformation and Preprocessing <br>
_tidy data & data spliting and preprocessing_

In [24]:
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Define features and target
X = p2ploan_df.drop(columns=['not.fully.paid'])  # Features
y = p2ploan_df['not.fully.paid']                # Target

# Step 2: Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Initialize Dummy Classifier
dummy_clf = DummyClassifier(strategy="most_frequent")  # Replace with 'stratified' if needed

# Step 4: Fit the dummy model
dummy_clf.fit(X_train, y_train)

# Step 5: Evaluate the dummy model
y_pred = dummy_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Baseline Dummy Model Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))


Baseline Dummy Model Accuracy: 0.84
              precision    recall  f1-score   support

           0       0.84      1.00      0.91      1611
           1       0.00      0.00      0.00       305

    accuracy                           0.84      1916
   macro avg       0.42      0.50      0.46      1916
weighted avg       0.71      0.84      0.77      1916



/Users/yasminhassan/miniforge3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/yasminhassan/miniforge3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/yasminhassan/miniforge3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", le

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Step 1: Define features and target
X = p2ploan_df.drop(columns=['not.fully.paid'])  # Features
y = p2ploan_df['not.fully.paid']                # Target

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 2: Define numeric and categorical columns
numeric_features = [
    'int.rate', 'installment', 'annual.inc', 'loan_income_ratio', 'dti', 
    'fico', 'days.with.cr.line', 'revol.bal', 'revol.util', 
    'inq.last.6mths', 'delinq.2yrs', 'pub.rec'
]
categorical_features = ['purpose', 'loan_categories', 'risk_category']

# Step 3: Preprocessing
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                   # Scale numeric features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # Encode categorical features
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Step 4: Build the full pipeline with a model
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Apply preprocessing
    ('classifier', LogisticRegression())  # Replace with any model you prefer
])

# Step 5: Fit the pipeline
model_pipeline.fit(X_train, y_train)

# Step 6: Evaluate the model
y_pred = model_pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))


Model Accuracy: 0.84
              precision    recall  f1-score   support

           0       0.84      0.99      0.91      1611
           1       0.39      0.03      0.05       305

    accuracy                           0.84      1916
   macro avg       0.62      0.51      0.48      1916
weighted avg       0.77      0.84      0.78      1916



####  Model Building<br>
_Model selection, Hyperparameter Optimization, Model evaluation, test data prediction_

### Results and Discussion <br>

### Conclusion and Limitations<br>
_discuss main findings, implications and limiations_

# Reference
1. Consumer Financial Protection Bureau. (n.d.). *Borrower risk profiles: Student loans*. Retrieved November 20, 2024, from [https://www.consumerfinance.gov/data-research/consumer-credit-trends/student-loans/borrower-risk-profiles/](https://www.consumerfinance.gov/data-research/consumer-credit-trends/student-loans/borrower-risk-profiles/)
2. Equifax. (n.d.). *Credit score ranges.* Retrieved November 20, 2024, from [https://www.equifax.com/personal/education/credit/score/articles/-/learn/credit-score-ranges/](https://www.equifax.com/personal/education/credit/score/articles/-/learn/credit-score-ranges/)
3. Coşer, A., Maer-Matei, M. M., & Albu, C. (2019). PREDICTIVE MODELS FOR LOAN DEFAULT RISK ASSESSMENT. Economic Computation & Economic Cybernetics Studies & Research, 53(2).
3.Khandani, A. E., Kim, A. J., & Lo, A. W. (2010). Consumer credit-risk models via machine-learning algorithms. Journal of Banking & Finance, 34(11), 2767-2787.
4. myFICO. (n.d.). *What's in my FICO® Scores?* Retrieved November 20, 2024, from [https://www.myfico.com/credit-education/whats-in-your-credit-score](https://www.myfico.com/credit-education/whats-in-your-credit-score#:~:text=FICO%20Scores%20are%20calculated%20using,and%20credit%20mix%20(10%25)
   
